Import gluon and mxnet and pandas.

gluon is for machine learning.

pandas is to access data

In [1]:
from d2l import mxnet as d2l
from mxnet import gluon, autograd, init, npx, np
from mxnet.gluon import nn  # neural network package from gluon
import os
import pandas as pd
npx.set_np()

In [2]:
set1 = "./Set1/TRAIN/TRAIN.csv"
label1 = "./Set1/TRAIN/TRAIN_labels.csv"
dataset = pd.read_csv(set1)
dataset.info()
label = pd.read_csv(label1)
label.info()
X = np.array(dataset.values)
y = label["theta1"]  # you can just index by typing the name using pandas
X = X.astype('float32')  # correct value type
y = np.array(y.values)  # correct object type
y = y.astype('float32')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 3600 entries, (0|0) to (35|99)
dtypes: float64(3600)
memory usage: 2.7 GB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   eventNumber  100000 non-null  int64  
 1   theta1       100000 non-null  float64
 2   z1           100000 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 2.3 MB


In the data, each line is an event. There should be around 36 points for each event if it hits all the screens, but some times it only hit a few. But most of them are multiples of 6. For example, if the angle has a magnitude over 10, it tends to not hit all 36 planes. Also, 100% hit efficenty mean that the particle will hit the wire in each plane it it can. In the test cases, there are 1000 lines in the data set and 100 different labels, hence each line represents an event with has the label which is the angle. There are 3600 columes in both the testing and test data set, and these corresponds to the wire hit. The bias is 0 since all points starts from z=0. 

In [3]:
x = X > 0
x.sum(axis=1)

array([36, 36, 36, ..., 36, 36, 36], dtype=int64)

Taking a look at the event 0, the second event, there are 3600 different possible wires to be hit. So there aer 3600 different veriables. One particle was shot out randomly and only 36 of these veriables are used. The theta value it suppose to give is 3.315625.

In [4]:
event1 = X[1,:]
event1index = event1 > 0
event1data = event1[event1index]
event1data

array([139.56125 , 183.91722 , 234.99887 , 195.3385  , 152.4103  ,
       108.72551 , 108.705605, 155.46568 , 195.71147 , 240.5206  ,
       227.01334 , 143.65202 , 165.15231 , 211.10278 , 249.0032  ,
       221.25485 , 170.24008 , 132.26541 , 138.36244 , 170.26302 ,
       238.37743 , 198.99002 , 156.37135 , 116.507904, 105.      ,
       146.70528 , 188.26672 , 227.94113 , 238.3008  , 142.6536  ,
       155.70299 , 201.31061 , 249.03552 , 220.23488 , 177.63501 ,
       139.6598  ])

In [5]:
y[0]

array(3.315625)

STEP 1. Read data in batches

In [6]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """Construct a Gluon data loader."""
    dataset = gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 1000
data_iter = load_array((X, y), batch_size)

In the first step, we use the data module to read random data. The object takes one or more tensors as arguments, this we gave the dataset and the labels. We used ArrayDataset to instantiate a DataLoader which takes a batch_size and shuffle Boolean to deterimind what data to use. Here we take a look at the data

In [7]:
next(iter(data_iter))

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([-9.539323e+00,  3.578473e+00,  2.950167e+00, -5.986791e+00,
         4.755381e+00,  6.870640e+00,  3.519502e+00, -3.555228e+00,
         5.560397e+00, -9.405040e+00,  8.423130e-01,  7.752433e+00,
         2.957897e+00,  8.927818e+00, -7.641076e+00,  5.819589e+00,
         5.807699e+00,  6.755280e+00, -4.759973e+00, -7.904869e+00,
         4.533058e+00, -6.474874e+00, -5.051648e+00,  1.639388e+00,
         3.193530e+00,  7.501271e+00,  7.662933e+00, -6.137539e+00,
         4.842704e+00,  2.986750e+00, -8.917894e+00,  3.408415e+00,
        -2.183505e+00, -3.365098e+00, -1.830980e-01,  7.633705e+00,
         2.659397e+00, -4.346369e+00,  2.246351e+00, -7.165988e+00,
         6.670464e+00,  6.850425e+00,  2.039974e+00,  7.518105e+00,
   

STEP 3. Defining the Model

We will use nn (neural networks) from gluon to define a model. the model veriable is net which is an instance of Sequential class. Sequential class if a container for several layers. We only need one layer, but if we had more layers, Sequential would just pass it the first layer and then pass the output as the second layer's input and so forth. 

In [8]:
net = nn.Sequential()
net.add(nn.Dense(1))  # Maybe I can change this to float32

We will want a fully-connected layer because each of the 3600 inputs are connected to the output. This is defined in the Dense class. We only want a single scalar output, so we set that number to 1.

STEP 3. Initializing Model Parameters

Using the initializer module from MXNet, we specify each weight from a randomly sampled normal distribution with mean 0 and standard deviation 0.01. The bias parameter will be initialized to 0 by defeult. Note that we are initalizing parameters even thought Gluon does not know how manny dimensions the input will have. This is because initializaion is actually deferred, hence we cannot access or manipulate them.

In [9]:
net.initialize(init.Normal(sigma=0.0001))

STEP 4. Defining the Loss Function

For linear regression, we implement the squared loss function L2LOSS. The loss function gives a value which tells us how much the estimated value is close to the real value. This will then be used for optimzation later.

In [10]:
from mxnet.gluon import loss as gloss
loss = gloss.L2Loss()

STEP 5. Defining the Optimzation Algorithm

We will use Minibatch Stochastic Gradient Descent (Minibatch SGD) as the tool to optimize this neural network. This is throught Gluon Trainer class. Our parameters are from net and we obtain them from net.collect_params(). The optimization algorithm is (sgd). The hyper-parameters, parameters we change, not of the dataset, is just the learning rate. The learning rate is just the step size of the weights. It is very low because if it is over 0.0001, then the loss function diverges and and code breaks down.

In [84]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.000008})

STEP 6. Training

Now time for training. 

In [85]:
X1, y1 = next(iter(data_iter))
net(X1)  # Gets the weights
l = loss(net(X1), y1)
print('loss: %f' % (l.mean().asnumpy()))

loss: 0.000344


In [86]:
l = loss(net(X), y)
print('loss: %f' % (l.mean().asnumpy()))

loss: 0.000387


In [87]:
num_epochs = 20
for epoch in range(1, num_epochs + 1):
    for X1, y1 in data_iter:
        with autograd.record():  # stores a function so that we can compute gradients later.
            l = loss(net(X1), y1)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(X), y)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 0.000384
epoch 2, loss: 0.000384
epoch 3, loss: 0.000383
epoch 4, loss: 0.000384
epoch 5, loss: 0.000384
epoch 6, loss: 0.000383
epoch 7, loss: 0.000382
epoch 8, loss: 0.000382
epoch 9, loss: 0.000382
epoch 10, loss: 0.000381
epoch 11, loss: 0.000381
epoch 12, loss: 0.000380
epoch 13, loss: 0.000381
epoch 14, loss: 0.000380
epoch 15, loss: 0.000380
epoch 16, loss: 0.000380
epoch 17, loss: 0.000379
epoch 18, loss: 0.000379
epoch 19, loss: 0.000379
epoch 20, loss: 0.000379


STEP 7. Get prediction

In [99]:
w = net[0].weight.data()
w.shape

(1, 3600)

In [16]:
X.shape

(100000, 3600)

In [89]:
sol=np.dot(X, w.T)

In [90]:
sol[1]

array([7.8954797])

In [93]:
sol[20]

array([-7.714302])

In [98]:
sol.shape

(100000, 1)

In [118]:
file_name = "net.params"
net.save_parameters('./Set1/TRAIN/model')